In [1]:
#Import
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
from PIL import Image
#Inception preprocessing code from https://github.com/tensorflow/models/blob/master/slim/preprocessing/inception_preprocessing.py
#useful to maintain training dimension
#from utils import inception_preprocessing
from preprocessing import inception_preprocessing
import sys

#from inception import inception
'''
Uso di slim e nets_factory (come per SLIM Tensorflow https://github.com/tensorflow/models/blob/master/slim/train_image_classifier.py)
per il ripristino della rete. 

Le reti devono essere censite in nets_factory (v. struttura file nella directory di questo notebook)
'''

slim = tf.contrib.slim
from nets import nets_factory, inception_v4

In [2]:
SAMPLE_IMAGE_NET_ROOT_DIR ='/var/ifs/data/hadoop-cloudera5/notebookDir/HUB/lgrazioli/ILSVRC Analysis/SMALL_ILSVRC/'
IMAGE_DF_PATH = SAMPLE_IMAGE_NET_ROOT_DIR + 'images_df.csv'
ENI_ROOT_DIR_LOGOS = '/var/ifs/data/Eni_logo/'
TF_RECORD_SIZE = 128

#Global Variables
IMAGE_NET_ROOT_PATH = '/var/ifs/data/tiny-imagenet-200/'
#IMAGE_NET_ROOT_PATH = '/data/lgrazioli/'
IMAGE_NET_LABELS_PATH = IMAGE_NET_ROOT_PATH + 'words.txt'
IMAGE_NET_TRAIN_PATH = IMAGE_NET_ROOT_PATH + 'train/'
TRAINING_CHECKPOINT_DIR = '/tmp/ImageNetTrainTransfer'
#Transfer learning CHECKPOINT PATH
#File ckpt della rete
CHECKPOINT_PATH = '/var/ifs/data/model-zoo/inceptionv4/tensorflow-1.2/inception_v4.ckpt'

In [3]:
images_df = pd.read_csv(IMAGE_DF_PATH)
#Eni Images
eni_paths = []
eni_labels = []
for eni_img in os.listdir(ENI_ROOT_DIR_LOGOS):
    eni_paths.append(ENI_ROOT_DIR_LOGOS + eni_img)
    eni_labels.append('ENI')
    
full_images_paths = list(images_df['path']) + eni_paths
full_images_labels = list(images_df['label']) + eni_labels

full_images_df = pd.DataFrame(full_images_paths, columns=['path'])
full_images_df['label'] = full_images_labels
full_images_df = full_images_df.sample(frac=1).reset_index(drop=True)
full_images_df.head(10)

,path,label
0,/ILSVRC/Data/CLS-LOC/train/n03075370/n03075370...,n03075370
1,/ILSVRC/Data/CLS-LOC/train/n04081281/n04081281...,n04081281
2,/ILSVRC/Data/CLS-LOC/train/n04081281/n04081281...,n04081281
3,/ILSVRC/Data/CLS-LOC/train/n04044716/n04044716...,n04044716
4,/ILSVRC/Data/CLS-LOC/train/n01698640/n01698640...,n01698640
5,/ILSVRC/Data/CLS-LOC/train/n04044716/n04044716...,n04044716
6,/ILSVRC/Data/CLS-LOC/train/n04081281/n04081281...,n04081281
7,/ILSVRC/Data/CLS-LOC/train/n02992529/n02992529...,n02992529
8,/ILSVRC/Data/CLS-LOC/train/n04131690/n04131690...,n04131690
9,/ILSVRC/Data/CLS-LOC/train/n01728920/n01728920...,n01728920


In [4]:
#Convert to list
path_list = list(full_images_df['path'])
label_list = list(full_images_df['label'])

#Replace Paths with SIMPLE ILSVRC
for idx,path in enumerate(path_list):
    path_list[idx] = path.replace('/ILSVRC/', SAMPLE_IMAGE_NET_ROOT_DIR)
    
#Construct label dictionary 
label_dict = {}
for idx, value in enumerate(set(label_list)):
    label_dict[value] = idx 
label_indices = [label_dict[label] for label in label_list]
num_classes = len(label_dict)
#Len of path must be the same of label len
assert len(label_indices) == len(path_list)




In [5]:
#Nonostante le annotations non tutte le immagini sono a colori
uncorrect_images = 0
full_color_path_images = []
full_color_labels = []
for idx,p in enumerate(path_list):
    im = np.array(Image.open(p))
    if im.shape[-1] == 3:
        full_color_path_images.append(path_list[idx])
        full_color_labels.append(label_indices[idx])
    else:
        uncorrect_images += 1
    if uncorrect_images % 10 == 0:
        sys.stdout.write("\r"+str(uncorrect_images))
        sys.stdout.flush()

10

IndexError: tuple index out of range

In [6]:
checkpoint_df = pd.DataFrame(full_color_path_images,columns=['path'])
checkpoint_df['labels'] = full_color_labels
checkpoint_df.to_csv('checkpoint_df.csv')

In [5]:
checkpoint_df = pd.read_csv('checkpoint_df.csv')
full_color_path_images = list(checkpoint_df['path'])
full_color_labels = list(checkpoint_df['labels'])

In [6]:
label_dict

{'ENI': 4,
 'n01698640': 7,
 'n01728920': 2,
 'n01729977': 0,
 'n02071294': 1,
 'n02672831': 6,
 'n02992529': 5,
 'n03075370': 10,
 'n04044716': 3,
 'n04081281': 9,
 'n04131690': 8}

In [7]:
def int64_feature(values):
    """Returns a TF-Feature of int64s.
    Args:
        values: A scalar or list of values.
    Returns:
        a TF-Feature.
    """
    
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def float_feature(values):
    return tf.train.Feature(float_list=tf.train.FloatList(value=values))

def bytes_feature(values):
    """Returns a TF-Feature of bytes.
    Args:
        values: A string.
    Returns:
        a TF-Feature.
    """
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))


def bottlenecks_to_tfexample(bottlenecks, labels, paths):
    return tf.train.Example(features=tf.train.Features(feature={
      'bottlenecks/encoded': float_feature(bottlenecks),
      'bottlenecks/label': int64_feature(labels),
      'bottlenecks/paths': bytes_feature(paths)  
    }))


In [8]:
output_tfrecord_filename = 'inception_bootlenecks_batch{0}.tfrecord'

In [9]:
EPOCHS = 1
BATCH_SIZE = 128
#Serve per capire quando il generatore è passato a batch appartenenti a una nuova epoca 
BATCH_PER_EPOCH = np.ceil(len(path_list) / BATCH_SIZE)

def parse_single_image(filename_queue):
    #Dequeue a file name from the file name queue
    #filename, y = filename_queue.dequeue()
    #Non bisogna invocare il dequeue il parametro della funziona è già lo scodamento
    filename, y_numeric = filename_queue[0], filename_queue[1]
    #A y manca solo il one-hot
    #y = tf.one_hot(y_numeric, num_classes)
    #y = y_numeric
    #Read image
    raw = tf.read_file(filename)
    #convert in jpg (in GPU!)
    jpeg_image = tf.image.decode_jpeg(raw)
    #Preprocessing with inception preprocessing
    #jpeg_image = tf.image.resize_images(jpeg_image, [300, 300], method=tf.image.ResizeMethod.BILINEAR)
    #Inception v4 default image size is 299
    jpeg_image = inception_preprocessing.preprocess_for_eval(jpeg_image, 299, 299)
    jpeg_image = tf.reshape(jpeg_image, [299, 299, 3])
    
    #jpeg_image = inception_preprocessing.preprocess_image(jpeg_image, 300, 300, is_training=False)
    return jpeg_image, y_numeric, filename
#jpeg_image = parse_single_image(filename_queue)

def get_batch(filenames, labels, batch_size, num_epochs=None):
    
    #Coda lettura file, slice_input_producer accetta una lista di liste (stessa dimensione)
    #Risultato dello scodamento è l'elemento corrente di ciascuna delle liste
    #Le liste sono rispettivamente la lista di file e la lista dei label
    filename_queue = tf.train.slice_input_producer([filenames, labels])
    
    #Lettura singolo record
    #jpeg_image,y,filename,y_numeric = parse_single_image(filename_queue)
    jpeg_image,y,filename = parse_single_image(filename_queue)
    
    # min_after_dequeue defines how big a buffer we will randomly sample
    #   from -- bigger means better shuffling but slower start up and more
    #   memory used.
    # capacity must be larger than min_after_dequeue and the amount larger
    #   determines the maximum we will prefetch.  Recommendation:
    #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 10
    capacity = min_after_dequeue + 3 * batch_size
    
    #tensors è la lista dei tensori delle single feature e immagini. Esegue batch_size volte i tensori example e label per ottenere il batch
    #num_threads incrementa effettivamente l'utilizzo della CPU (confermato dal throughput visisible sul cloudera manager,
    #resta comunque un throughput lento ....
    example_batch = tf.train.shuffle_batch(
        tensors=[jpeg_image, y,filename], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue, allow_smaller_final_batch=True, num_threads=3)
    
    return example_batch

In [10]:
with tf.device('/cpu:0'):
    with tf.name_scope('preprocessing') as scope:
        #x,y,filenames,y_numeric = get_batch(full_color_path_images, full_color_labels, batch_size=BATCH_SIZE)
        x,y,filenames = get_batch(full_color_path_images, full_color_labels, batch_size=BATCH_SIZE)

        #x = tf.contrib.layers.flatten(x)
        #flatten_x = tf.contrib.layers.flatten(x) 
        
with tf.device('/gpu:0'):
    #inception prelogits 
    inception_v4.inception_v4(x,is_training=False, dropout_keep_prob=1, create_aux_logits=False)
    #prelogits = tf.placeholder(tf.float32, [None, 1536], name='prelogits_placeholder')
    inception_logits = tf.get_default_graph().get_tensor_by_name("InceptionV4/Logits/PreLogitsFlatten/Reshape:0")
    
init = tf. global_variables_initializer()

In [11]:
total_batch = int(np.ceil(len(full_color_path_images) / BATCH_SIZE))
total_batch

40

In [12]:
#GPU config
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
#Saver per restoring inception net
saver = tf.train.Saver()

with tf.Session(config=config) as sess:
    sess.run(init)
    sess.run(tf.local_variables_initializer())
    
    #Start populating the filename queue.
    coord = tf.train.Coordinator()
    #Senza questa chiamata non partono i thread per popolare la coda che permette di eseguire la read
    threads = tf.train.start_queue_runners(coord=coord)
    
    current_batch = 0
    while True:
        #x_batch, y_batch = sess.run([x,y])
        #print(x_batch.shape)
    
        batch_logits, batch_y, batch_filenames = sess.run([inception_logits,y,filenames])

        with tf.python_io.TFRecordWriter(output_tfrecord_filename.format(current_batch)) as tf_record_writer:
            print("Producing {0} tfrecord".format(output_tfrecord_filename.format(current_batch)))
            example = bottlenecks_to_tfexample(bottlenecks=[float(x) for x in list(batch_logits.flatten('F'))],
                                               labels=list(batch_y.flatten('F')), 
                                               paths=list(batch_filenames.flatten('F'))
                                              )
            tf_record_writer.write(example.SerializeToString())
        #print(batch_logits.shape)
        #print(batch_y.shape)
        #print(batch_filenames.shape)
        current_batch += 1
        if current_batch >= total_batch:
            break
    coord.request_stop()
    coord.join(threads)
    sess.close()

<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch0.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch1.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch2.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch3.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch4.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch5.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch6.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch7.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch8.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch9.tfrecord tfrecord
<class 'numpy.ndarray'>
(196608,)
Producing incpetion_bootlenecks_batch10.tfrecord tfrecor